# Recreating CNN's Fear and Greed Index

CNN uses some proprietary indicators to attempt to sense how fearful or greedy investors are at a given time. The results are then displayed on a scale of 0 to 100, where 0 is most fearful, and 100 is most greedy. If investors are greedy, then prices should rise, whereas if investors are fearful, prices should fall.

A simple idea would be to buy or hold when the index is above 50 and sell or hold when the index is below 50.

A more complex idea would be to apply mean reversion when bullish, and momentum reversal when bearish, while referencing external price data. 

### Fear and Greed Factors

According to CNN, their index is based on the following factors:
    
1. Price breadth: the volume of shares trading on the rise versus those declining.
2. Price strength: the number of securities hitting 52-week highs and lows on the NYSE.
3. Junk bond demand: the spread between yields on investment grade and junk bonds.
4. Market volatility: the VIX.
5. Put and call ratio: the volume of calls relative to puts.
6. Safe haven demand: the difference in returns for securities versus treasuries.
7. Price momentum: the SPX versus its 125-day moving average.

In [71]:
import re
import requests

In [158]:
msc = requests.request('GET', 'https://www.mcoscillator.com/market_breadth_data/')
msc.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"\n        "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">\n\n<head>\n\t\t<meta http-equiv="content-type" content="text/html; charset=utf-8" />\n        <meta name="msapplication-config" content="none"/>\n\n\t<!--[if IE]><![endif]--> <!--prevents blocking of css IN IE8 --> \n\t<title>Stock Market Breadth Data | Daily Oscillator Data</title>\n\t\t<!--[if IE]><![endif]-->\n        <link rel="stylesheet" type="text/css" href="/css/main_carryover.css" media="screen"/>\n\t<link rel="stylesheet" type="text/css" href="/css/print.css" media="print" />\n\n        <link href="/images/touch-icon/apple-touch-icon-precomposed.png" rel="apple-touch-icon-precomposed" />\n        <link href="/images/touch-icon/apple-touch-icon-72x72-precomposed.png" rel="apple-touch-icon-precomposed" sizes="72x72" />\n        <link href="/images/touch-icon/apple-touch-icon-114x1

In [165]:
msc_value = re.sub('\D', '', re.search('SUMM Index</th><td>\n\d+\n</td><td class="decimal">.\n\d+', msc.text).group(0))
msc_value

'2158623'

In [72]:
# Scrape the data from CNN's Fear and Green Index homepage.
response = requests.request("GET", 'http://money.cnn.com/data/fear-and-greed/')

In [87]:
# Parse the body using RegEx.
fngi = {
    'now': re.sub('\D', '', re.search('Now: \d+', response.text).group(0)),
    'weekly': re.sub('\D', '', re.search('Week Ago: \d+', response.text).group(0)),
    'monthly': re.sub('\D', '', re.search('Month Ago: \d+', response.text).group(0)),
    'yearly': re.sub('\D', '', re.search('Year Ago: \d+', response.text).group(0))
}

fngi

{'now': '59', 'weekly': '53', 'monthly': '59', 'yearly': '47'}

### Stock Breadth

We will be fetching the winners and losers from Yahoo Finance through RapidAPI as our universe. Since there is no way to qualify this data, we will just have to take their word for it for now.

Note: there is a possibility that using present day data will introduce survivorship bias.


In [136]:
# Fetch the data from Yahoo Finance.
url = "https://rapidapi.p.rapidapi.com/market/v2/get-movers"

# Pagination is possible here through optional start and count params.
querystring = {
    "region": "US",
    "lang": "en-US"
}

headers = {
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
    'x-rapidapi-key': "ecb945c1afmsh7c2c4b7ae799817p182789jsn66e955430b0f"
}

# Convert response to a JSON, and from a JSON to a sanitized list before we continue.
movers = requests.request("GET", url, headers=headers, params=querystring).json()['finance']['result']
movers

[{'id': 'ec5bebb9-b7b2-4474-9e5c-3e258b61cbe6',
  'title': 'Day Gainers - US',
  'description': 'Stocks ordered in descending order by price percent change greater than 3% with respect to the previous close',
  'canonicalName': 'DAY_GAINERS',
  'criteriaMeta': {'size': 6,
   'offset': 0,
   'sortField': 'percentchange',
   'sortType': 'DESC',
   'quoteType': 'EQUITY',
   'topOperator': 'AND',
   'criteria': [{'field': 'percentchange',
     'operators': ['GT'],
     'values': [3.0],
     'labelsSelected': []},
    {'field': 'region',
     'operators': ['EQ'],
     'values': [],
     'labelsSelected': [53]},
    {'field': 'intradaymarketcap',
     'operators': ['EQ'],
     'values': [],
     'labelsSelected': [1, 2, 3]},
    {'field': 'dayvolume',
     'operators': ['GT'],
     'values': [15000],
     'labelsSelected': []}]},
  'rawCriteria': '{"offset":0,"size":6,"sortField":"percentchange","sortType":"DESC","quoteType":"EQUITY","query":{"operator":"AND","operands":[{"operator":"GT","op

In [153]:
# Grab the quotes of both daily winners and losers.
winners = next((x for x in movers if x['canonicalName'] == 'DAY_GAINERS'), None)['quotes']
losers = next((x for x in movers if x['canonicalName'] == 'DAY_LOSERS'), None)['quotes']

winners, losers

([{'language': 'US',
   'region': 'EN-US',
   'quoteType': 'EQUITY',
   'quoteSourceName': 'Delayed Quote',
   'triggerable': False,
   'firstTradeDateMilliseconds': 1260455400000,
   'priceHint': 2,
   'exchange': 'NYQ',
   'market': 'us_market',
   'fullExchangeName': 'NYSE',
   'marketState': 'REGULAR',
   'sourceInterval': 15,
   'exchangeDataDelayedBy': 0,
   'exchangeTimezoneName': 'America/New_York',
   'exchangeTimezoneShortName': 'EDT',
   'gmtOffSetMilliseconds': -14400000,
   'esgPopulated': False,
   'tradeable': True,
   'symbol': 'CIT'},
  {'language': 'US',
   'region': 'EN-US',
   'quoteType': 'EQUITY',
   'quoteSourceName': 'Delayed Quote',
   'triggerable': False,
   'firstTradeDateMilliseconds': -252322200000,
   'priceHint': 2,
   'exchange': 'NYQ',
   'market': 'us_market',
   'fullExchangeName': 'NYSE',
   'marketState': 'REGULAR',
   'sourceInterval': 15,
   'exchangeDataDelayedBy': 0,
   'exchangeTimezoneName': 'America/New_York',
   'exchangeTimezoneShortName':

In [155]:
# Grab the winning and losing symbols.
win_symbols = list(map(lambda x: x['symbol'], winners))
lose_symbols = list(map(lambda x: x['symbol'], losers))

win_symbols, lose_symbols

(['CIT', 'NAV', 'BMI', 'RYCEF', 'FLEX', 'RAZFF'],
 ['NKLA', 'IGMS', 'SLB', 'JBHT', 'SIUIF', 'GLPG'])